In [1]:
!pip install -q sentence-transformers spacy textblob
!python -m spacy download en_core_web_sm
!python -m textblob.download_corpora


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 

In [2]:
import spacy
from sentence_transformers import SentenceTransformer, util
import random

nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer('all-MiniLM-L6-v2')

!pip install transformers --quiet

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
!pip install transformers --quiet

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

llm_model = "google/flan-t5-small"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(llm_model)
llm_feedback = pipeline("text2text-generation", model=llm_model, tokenizer=llm_tokenizer)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [4]:
def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)


In [5]:
questions = {
    "data analyst": [
        {
            "id": "q1",
            "question": "How do you approach data cleaning?",
            "ideal_answer": "I use pandas to handle missing values, remove duplicates, and standardize formats."
        },
        {
            "id": "q2",
            "question": "What is the difference between variance and standard deviation?",
            "ideal_answer": "Variance measures data spread; standard deviation is its square root."
        },
        {
            "id": "q3",
            "question": "Describe a project where you used SQL.",
            "ideal_answer": "I wrote SQL queries to extract insights from customer data and created summary dashboards."
        }
    ],

    "hr": [
        {
            "id": "q1",
            "question": "Describe a time you handled conflict in a team.",
            "ideal_answer": "I encouraged open communication, mediated differences, and aligned everyone on shared goals."
        },
        {
            "id": "q2",
            "question": "What steps do you take in recruitment?",
            "ideal_answer": "I handle sourcing, screening, interviewing, and onboarding candidates efficiently."
        },
        {
            "id": "q3",
            "question": "How do you ensure diversity and inclusion in hiring?",
            "ideal_answer": "By using blind resumes, structured interviews, and broadening sourcing strategies."
        }
    ],

    "software engineer": [
        {
            "id": "q1",
            "question": "Explain the difference between multithreading and multiprocessing.",
            "ideal_answer": "Multithreading shares the same memory space; multiprocessing runs processes in separate memory spaces for parallelism."
        },
        {
            "id": "q2",
            "question": "How do you handle version control?",
            "ideal_answer": "I use Git for branching, merging, and managing code across teams."
        },
        {
            "id": "q3",
            "question": "Describe your experience with REST APIs.",
            "ideal_answer": "I have designed and consumed REST APIs using Flask and FastAPI, ensuring stateless communication."
        }
    ],

    "product manager": [
        {
            "id": "q1",
            "question": "How do you prioritize product features?",
            "ideal_answer": "I use frameworks like RICE and MoSCoW based on customer impact, effort, and alignment with strategy."
        },
        {
            "id": "q2",
            "question": "Describe a time you handled conflicting stakeholder requirements.",
            "ideal_answer": "I conducted meetings to understand concerns, aligned on goals, and proposed a roadmap balancing priorities."
        },
        {
            "id": "q3",
            "question": "What’s your approach to user research?",
            "ideal_answer": "I gather insights through surveys, user interviews, A/B testing, and analytics tools."
        }
    ],

    "marketing analyst": [
        {
            "id": "q1",
            "question": "How do you measure the effectiveness of a marketing campaign?",
            "ideal_answer": "By analyzing KPIs like ROI, CTR, conversion rate, and customer acquisition cost using tools like Google Analytics."
        },
        {
            "id": "q2",
            "question": "What tools do you use for marketing data analysis?",
            "ideal_answer": "Google Analytics, HubSpot, Excel, and Tableau are my go-to tools for analyzing campaign performance."
        },
        {
            "id": "q3",
            "question": "How do you segment a market?",
            "ideal_answer": "I segment based on demographics, psychographics, behavior, and customer needs using collected data."
        }
    ]
}


In [6]:
available_roles = list(questions.keys())

print("👋 Welcome to the AI Interview Bot!")
print("Please choose a role to apply for:")

for idx, role in enumerate(available_roles, start=1):
    print(f"{idx}. {role.title()}")

role_choice = int(input("Enter your choice (number): "))
user_role = available_roles[role_choice - 1]


👋 Welcome to the AI Interview Bot!
Please choose a role to apply for:
1. Data Analyst
2. Hr
3. Software Engineer
4. Product Manager
5. Marketing Analyst
Enter your choice (number): 1


In [7]:
role_questions = random.sample(questions[user_role], k=min(2, len(questions[user_role])))

user_answers = {}

for q in role_questions:
    print(f"\n💬 {q['question']}")
    user_input = input("Your Answer: ")
    user_answers[q['id']] = {
        "user_answer": user_input,
        "ideal_answer": q["ideal_answer"]
    }



💬 How do you approach data cleaning?
Your Answer: “I approach data cleaning by first understanding the dataset’s structure and identifying common issues like missing values, duplicates, inconsistent formatting, or outliers. I then handle missing data using appropriate methods such as imputation or removal, normalize formats (like dates or text case), and use libraries like Pandas to automate checks and transformations. This ensures the data is clean, consistent, and ready for analysis or modeling.”

💬 Describe a project where you used SQL.
Your Answer: “In one project, I used SQL to analyze sales data from a retail database. I wrote queries to extract monthly revenue trends, customer purchase behavior, and product-wise sales performance. I used JOIN operations to combine customer and transaction tables, applied filters with WHERE, and aggregated data using GROUP BY and SUM. This helped generate insights for the marketing team to optimize campaigns.”


In [8]:
def get_similarity(text1, text2):
    emb1 = model.encode(text1, convert_to_tensor=True)
    emb2 = model.encode(text2, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2)[0][0])


In [9]:
def get_feedback(score):
    if score >= 0.8:
        return "✅ Excellent answer — very relevant and complete."
    elif score >= 0.5:
        return "🟡 Decent answer — try to elaborate more or add examples."
    else:
        return "🔴 Needs improvement — try to be more specific or relevant."


In [10]:
def get_llm_feedback(answer):
    prompt = f"Give constructive interview feedback on the following answer: '{answer}'"
    result = llm_feedback(prompt, max_length=100, do_sample=False)
    return result[0]['generated_text']


In [11]:
results = {}

for qid, pair in user_answers.items():
    cleaned_user = preprocess(pair["user_answer"])
    cleaned_ideal = preprocess(pair["ideal_answer"])

    sim_score = get_similarity(cleaned_user, cleaned_ideal)
    score_5 = round(sim_score * 5, 2)
    feedback = get_feedback(sim_score)

    llm_response = get_llm_feedback(pair["user_answer"])

results[qid] = {
    "user_answer": pair["user_answer"],
    "similarity_score": round(sim_score, 3),
    "score_out_of_5": score_5,
    "feedback": feedback,
    "llm_feedback": llm_response
}


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [12]:
import pandas as pd
print("\n📊 Final Interview Results:\n")
pd.DataFrame(results).T[["user_answer", "similarity_score", "score_out_of_5", "feedback", "llm_feedback"]]



📊 Final Interview Results:



,user_answer,similarity_score,score_out_of_5,feedback,llm_feedback
q3,"“In one project, I used SQL to analyze sales d...",0.555,2.77,🟡 Decent answer — try to elaborate more or add...,JOIN operations to analyze sales data from a r...


In [13]:
import json
with open("interview_results.json", "w") as f:
    json.dump(results, f, indent=2)
